<a href="https://colab.research.google.com/github/sofiaaic/PrediccionTendenciasPoliticasChile/blob/joaquin/notebooks/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Acá dejaremos el codigo para extraer los datos RAW.

In [3]:
# (Opcional) instala motores para Excel si hace falta
!pip -q install openpyxl


In [4]:
# 1) Montar Drive
from google.colab import drive

CARPETA = '/content/drive/MyDrive/data_votantes_chile'  # ajusta a tu ruta
drive.mount('/content/drive')  # autoriza con tu cuenta

Mounted at /content/drive


In [ ]:

# 2) Configurar ruta de carpeta en tu Drive
#    Ejemplo: "Mi unidad/Proyectos/Excels"
import os, glob
import pandas as pd

CARPETA = '/content/drive/MyDrive/data_votantes_chile'  # ajusta a tu ruta


# 3) Buscar archivos .xlsx (evitar temporales ~$.xlsx)
rutas = [p for p in glob.glob(os.path.join(CARPETA, '**', '*.xlsx'), recursive=True)
         if not os.path.basename(p).startswith('~$')]

print(f'Encontrados {len(rutas)} archivos:')
for r in rutas: print('-', os.path.relpath(r, CARPETA))

# 4) Leer y concatenar (elige la hoja, columnas, tipos, etc.)
dfs = []
for ruta in rutas:
    print("Ruta: ", ruta)
    # Personaliza: sheet_name, usecols, dtype, skiprows…
    df = pd.read_excel(
        ruta,
        sheet_name='Votación en Chile',
        skiprows=6,
        engine='openpyxl'
    )
    df['__archivo_origen'] = os.path.basename(ruta)  # traza
    print(df.head())
    dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)
df_total.head()


Encontrados 10 archivos:
- 2021_12_Presidencial_2V_Datos_Eleccion.xlsx
- 2021_11_Presidencial_1V_Datos_Eleccion.xlsx
- 2021_07_PresidencialPrimarias_Datos_Eleccion.xlsx
- 2017_12_Presidencial_2V_Datos_Eleccion.xlsx
- 2017_11_Presidencial_Datos_Eleccion.xlsx
- 2017_07_PrimariaPresidencial_DatosEleccion.xlsx
- 2013_11_Presidencial_DatosEleccion.xlsx
- 2013_12_Presidencial_2V_DatosEleccion.xlsx
- 2013_06_PresidencialPrimarias_DatosEleccion.xlsx
- 2025_06_PrimariasPresidencial_Datos_Eleccion.xlsx
Ruta:  /content/drive/MyDrive/data_votantes_chile/2021_12_Presidencial_2V_Datos_Eleccion.xlsx
   Nro Región       Región    Circunscripción senatorial    Distrito  \
0           1  DE TARAPACA  CIRCUNSCRIPCION SENATORIAL 2  DISTRITO 2   
1           1  DE TARAPACA  CIRCUNSCRIPCION SENATORIAL 2  DISTRITO 2   
2           1  DE TARAPACA  CIRCUNSCRIPCION SENATORIAL 2  DISTRITO 2   
3           1  DE TARAPACA  CIRCUNSCRIPCION SENATORIAL 2  DISTRITO 2   
4           1  DE TARAPACA  CIRCUNSCRIPCION SENA